In [ ]:

%pprint
import sys
if ('../py' not in sys.path): sys.path.insert(1, '../py')

In [ ]:

import os.path as osp
shared_folder = osp.abspath(osp.join('..', '..', 'share'))
if shared_folder not in sys.path: sys.path.insert(1, shared_folder)
from notebook_utils import NotebookUtilities
nu = NotebookUtilities(
    data_folder_path=osp.abspath(osp.join(
        '..', 'data'
    )),
    saves_folder_path=osp.abspath(osp.join(
        '..', 'saves'
    ))
)

In [1]:

%pprint
import sys
if ('../py' not in sys.path): sys.path.insert(1, '../py')
from jobpostlib import (cu, datetime, duration, freq, hau, humanize, nu, scrfcu, slrcu, su, t0, time, wsu, speech_engine)
import os

Pretty printing has been turned OFF


ModuleNotFoundError: No module named 'jobpostlib'

In [21]:

from pandas import DataFrame

button_css = '#js-btn-save > button:nth-child(2)'
table_css = '.bg-color--white'
cookies_accepted = False
driver = wsu.get_driver(verbose=False); speech_engine.say('Browser available'); speech_engine.runAndWait()

In [116]:

import re
from np import nan

parts_list = [
    7707, 7262, 6190, 6296, 1713, 4790, 6567, 7310, 7090, 6815, 7235, 6107, 1356, 6799, 6516, 6110, 6835, 6854, 1265, 6818, 6815, 6115, 4491, 6799, 3538, 7475, 4594, 7101, 4733, 30432, 7200, 7203,
    8547, 7238, 3545, 3051, 7251, 6155, 7655, 7121, 6834, 6712, 6150, 7313, 3017, 7115, 7670, 7700, 6045, 7414, 1788, 7128, 6664, 6555, 6038, 5938, 4093, 7134, 7130, 7127, 3579, 7944, 7701, 4793, 4794
]
inventory_regex = re.compile(r'(\S+) \(Inv\)')

In [152]:

rows_list = []
for part_name in parts_list:
    inventory_url = f'https://www.bricklink.com/catalogItemInv.asp?S={part_name}-1'
    wsu.driver_get_url(driver, inventory_url, verbose=False)
    inventory_soup = nu.get_page_soup(driver.page_source)
    set_css = '#id-main-legacy-table > tbody:nth-child(1) > tr:nth-child(1) > td:nth-child(1) > table:nth-child(4) > tbody:nth-child(1) > tr:nth-child(1) > td:nth-child(1) > center:nth-child(1) > font:nth-child(1) > b:nth-child(1)'
    set_name = inventory_soup.select(set_css)[0].text
    set_url = inventory_soup.select('#img-1')[0]['src']
    for table_soup in inventory_soup.find_all(name='table', attrs={'class': 'ta', 'width': '100%'}):
        for script in table_soup.find_all('script'):
            script.extract()
        for tr_soup in table_soup.find_all(name='tr', attrs={'class': True, 'bgcolor': True}):
            for img_soup in tr_soup.find_all(name='img', attrs={'src': True, 'alt': True}):
                img_src = img_soup['src']
                tds_list = tr_soup.find_all(name='td')
                item_description = tds_list[3].find_all(name='b')[0].text.strip()
                item_quantity = int(tds_list[1].text.strip())
                item_url = 'https://www.bricklink.com' + tds_list[2].find_all(name='a')[0]['href']
                item_number = tds_list[2].find_all(name='a')[0].text.strip()
                row_dict = {
                    'part_name': part_name, 'search_url': f'https://www.bricklink.com/catalogList.asp?pg=1&q={part_name}&catLike=W&sortBy=N&sortAsc=A&v=0', 'inventory_url': inventory_url,
                    'set_name': set_name, 'set_url': set_url, 'img_src': img_src, 'item_description': item_description, 'item_quantity': item_quantity,
                    'item_url': item_url, 'item_number': item_number
                }
                rows_list.append(row_dict)
lego_inventory_df = DataFrame(rows_list)
nu.store_objects(lego_inventory_df=lego_inventory_df)

Pickling to C:\Users\daveb\OneDrive\Documents\GitHub\job-hunting\saves\pkl\lego_inventory_df.pkl


In [153]:

from IPython.display import HTML

min_part = lego_inventory_df.groupby('part_name').size().sort_values().index[0]
mask_series = (lego_inventory_df.part_name == min_part)
columns_list = ['img_src', 'item_quantity']
html_str = lego_inventory_df[mask_series][columns_list].head(6).T.to_html()
html_str = re.sub(r'\s+<th>(|img_src|item_quantity)</th>', '', html_str)
html_str = re.sub('<td>https:', '<td><img src="https:', html_str)
html_str = re.sub(r'\.(jpg|gif)</td>', r'.\1" /></td>', html_str)
HTML(html_str)

1347,1348,1349,1350,1351,1352
,,,,,
1,1,2,1,1,1


In [156]:

for (part_name, set_name, set_url), set_name_df in lego_inventory_df.groupby(['part_name', 'set_name', 'set_url']):
    set_html = f'<figure style="text-align:center;"><img src="{set_url}" alt="Set No: {part_name}  Name: {set_name}" /><figcaption>{set_name}</figcaption></figure>'
    set_html += '<br /><table><tr>'
    item_htmls_list = []
    for (img_src, item_number, item_description, item_quantity), item_name_df in set_name_df.groupby(['img_src', 'item_number', 'item_description', 'item_quantity']):
        item_html = f"""<figure><img src="{img_src}" alt="Part No: {item_number}  Name: {item_description}"><figcaption>{item_quantity}</figcaption></figure>"""
        item_htmls_list.append(item_html)
    num_groups = 6
    
    # Calculate the size of each group
    group_size = len(item_htmls_list) // num_groups
    
    # Handle any remainder
    remainder = len(item_htmls_list) % num_groups
    
    grouped_items = []
    start = 0
    
    for i in range(num_groups):
        
        # Add one extra item to the group if there's a remainder
        end = start + group_size + (1 if i < remainder else 0)
        
        grouped_items.append(item_htmls_list[start:end])
        start = end
    
    # Add the groups as columns
    for group_list in grouped_items:
        set_html += '<td>' + '<br />'.join(group_list) + '</td>'
    
    set_html += '</tr></table>'
    file_name = f'set{part_name}.html'
    file_path = os.path.join(cu.SAVES_HTML_FOLDER, file_name)
    with open(file_path, 'w', encoding=nu.encoding_type) as f:
        f.write(set_html)

In [ ]:

table_dict = nu.get_page_tables(str(table_soup), verbose=False)[0].to_dict(orient='list')
if 'Names or topics' in table_dict.keys(): topics_list = table_dict['Names or topics']
elif 'Persons or topics' in table_dict.keys(): topics_list = table_dict['Persons or topics']
else: print(table_dict.keys()); raise
for date_str, topic_str in zip(table_dict['Date'], topics_list):
    cover_topic_str = re.sub(r', *Sr\.', '_ Sr.', cover_topic)
    cover_topic_str = re.sub(r', *Duke', '_ Duke', cover_topic_str)
    cover_topics_list = re.split(', *', cover_topic_str, 0)
    for cover_topic_str in cover_topics_list:
        cover_topic_str = cover_topic_str.split('[')[0].replace('_', ',')
        if (cover_topic_str == 'Queen Marie'): cover_topic_str = 'Marie of Romania'
        elif (cover_topic_str == 'Queen Marie'): cover_topic_str = 'Marie of Romania'

In [ ]:

import os

exe_list = ['wc', 'gvpr']
for sub_directory, directories_list, files_list in os.walk(r'D:\\'):
    for file_name in files_list:
        if any(map(lambda x: file_name == f'{x}.exe', exe_list)):
            file_path = os.path.join(sub_directory, file_name)
            print(f'{file_path}')

In [ ]:

%run ../load_magic/storage.py
%run ../load_magic/dataframes.py
%run ../load_magic/paths.py
%run ../load_magic/lists.py
%run ../load_magic/environment.py
%pprint
notebook_path = get_notebook_path()
print(notebook_path)
s = Storage()
print(['s.{}'.format(fn) for fn in dir(s) if not fn.startswith('_')])
[fn for fn in dir() if not fn.startswith('_')]

In [ ]:

                try:
                except Exception as e:
                    print(f'{e.__class__} exception at peakutils.indexes({y}, thres={thres}, min_dist={min_dist}): {str(e).strip()}')

In [17]:

rest_times_df = DataFrame([])
runners_dict = {
    'amanda': 'Amanda Nelson', 'bartosz': 'Bartosz Fudali', 'ihor': 'Ihor Verys', 'keith': 'Keith Russell', 'merijn': 'Merijn Geerts', 'phil': 'Phil Gore', 'sam': 'Sam Harvey',
    'terumichi': 'Terumichi Morishita'
}
for sub_directory, directories_list, files_list in os.walk(r'C:\Users\daveb\Downloads'):
    for file_name in files_list:
        if file_name.endswith('.html') and ('homepage' not in file_name):
            file_path = osp.join(sub_directory, file_name)
            rest_times_df = pd.concat(
                [rest_times_df, get_single_runner_rest_times_data_frame(table_path=file_path, runner_name=runners_dict[file_name.split('.')[0]])], axis='index'
            )
rest_times_df.sample(min(16, rest_times_df.shape[0])).dropna(axis='columns', how='all').T

,72,41,88,56,87,70,36,28,91,60,79,0,25,4,58,61
#,73,42,89,57,88,71,37,29,92,61,80,1,26,5,59,62
Measurement,79:47:58,48:43:22,95:44:59,63:55:28,94:48:10,77:47:34,43:40:44,35:50:34,98:49:57,67:46:47,86:52:13,7:53:36,32:53:33,11:37:07,65:56:52,68:39:58
Lap Split,47:58,43:22,44:59,55:28,48:10,47:34,40:44,50:34,49:57,46:47,52:13,53:36,53:33,37:06,56:52,39:58
Rest Time,721,997,900,271,709,745,1155,565,602,792,466,383,386,1373,187,1201
runner_name,Sam Harvey,Amanda Nelson,Ihor Verys,Amanda Nelson,Terumichi Morishita,Merijn Geerts,Keith Russell,Terumichi Morishita,Phil Gore,Bartosz Fudali,Bartosz Fudali,Ihor Verys,Ihor Verys,Bartosz Fudali,Merijn Geerts,Phil Gore


In [ ]:

print()
for i in range(0, 180, 20):
    print(f'''    <path
       class="tick-style major-tick"
       d="m {1.56 * i + 0.825315},161.4459 v 13.10819"
       id="path-millennium-tick{i:03}"
       style="stroke-opacity:1;stroke-width:0.79375;stroke:#160f20;"
    />''')
    # print(f'''    <path
    #    class="tick-style minor-tick"
    #    d="m {1.56 * i + 0.825315},163.31371 v 9.37257"
    #    id="path-quincentenary-tick{i:03}"
    #    style="stroke-opacity:1;stroke:#160f20;"
    # />''')
    # print(f'''    <path
    #    class="century-tick"
    #    d="m {1.56 * i + 0.825315},164.825 v 6.35"
    #    id="path-century-tick{i:03}" />''')


    <path
       class="tick-style major-tick"
       d="m 0.825315,161.4459 v 13.10819"
       id="path-millennium-tick000"
       style="stroke-opacity:1;stroke-width:0.79375;stroke:#160f20;"
    />
    <path
       class="tick-style major-tick"
       d="m 32.025315000000006,161.4459 v 13.10819"
       id="path-millennium-tick020"
       style="stroke-opacity:1;stroke-width:0.79375;stroke:#160f20;"
    />
    <path
       class="tick-style major-tick"
       d="m 63.22531500000001,161.4459 v 13.10819"
       id="path-millennium-tick040"
       style="stroke-opacity:1;stroke-width:0.79375;stroke:#160f20;"
    />
    <path
       class="tick-style major-tick"
       d="m 94.42531500000001,161.4459 v 13.10819"
       id="path-millennium-tick060"
       style="stroke-opacity:1;stroke-width:0.79375;stroke:#160f20;"
    />
    <path
       class="tick-style major-tick"
       d="m 125.62531500000001,161.4459 v 13.10819"
       id="path-millennium-tick080"
       style="stroke-opacity:1;s